### Creating Panels
A Panel is a three-dimensional analogue of DataFrame.  Each item (the analogue of columns in a DataFrame) in a Panel is a DataFrame.

The term panel data is derived
from econometrics and is partially responsible for the name pandas: pan(el)-da(ta)-s. The names for the 3 axes within a panel are intended to give some semantic meaning to describing operations involving panel data and, in particular, econometric
analysis of panel data. However, for the strict purposes of slicing and dicing a collection of DataFrame objects, you
may find the axis names slightly arbitrary:

- items: axis 0, each item corresponds to a DataFrame contained inside
- major_axis: axis 1, it is the index (rows) of each of the DataFrames
- minor_axis: axis 2, it is the columns of each of the DataFrames

source: http://pandas.pydata.org/pandas-docs/stable/dsintro.html#panel

In [1]:
import pandas as pd
import numpy as np
import datetime
from pandas_datareader import data, wb

pd.set_eng_float_format(accuracy=2, use_eng_prefix=True)

In [2]:
my_first_panel = pd.Panel(np.random.randn(2, 5, 4), 
                          items=['Item01', 'Item02'],
                          major_axis=pd.date_range('9/6/2016', periods=5),
                          minor_axis=['A', 'B', 'C', 'D'])
my_first_panel

<class 'pandas.core.panel.Panel'>
Dimensions: 2 (items) x 5 (major_axis) x 4 (minor_axis)
Items axis: Item01 to Item02
Major_axis axis: 2016-09-06 00:00:00 to 2016-09-10 00:00:00
Minor_axis axis: A to D

### From dict of DataFrame objects
Note that the values in the dict need only be convertible to DataFrame.

In [3]:
dictionary_of_data_frames = {'Item1' : pd.DataFrame(np.random.randn(4, 3)),
                             'Item2' : pd.DataFrame(np.random.randn(4, 2))}
my_dictionary_panel = pd.Panel(dictionary_of_data_frames)
my_dictionary_panel

<class 'pandas.core.panel.Panel'>
Dimensions: 2 (items) x 4 (major_axis) x 3 (minor_axis)
Items axis: Item1 to Item2
Major_axis axis: 0 to 3
Minor_axis axis: 0 to 2

##### Panel.from_dict()
One helpful factory method is Panel.from_dict, which takes a dictionary of DataFrames, and has the following named parameters:

| Parameter | Default | Description                                         |
|-----------|---------|-----------------------------------------------------|
| intersect | False   | drops elements whose indices do not align           |
| orient    | items   | use minor to use DataFrames’ columns as panel items |

Orient is especially useful for mixed-type DataFrames. If you pass a dict of DataFrame objects with mixed-type columns, all of the data will get upcasted to dtype=object unless you pass orient='minor':

In [4]:
oriented_panel = pd.Panel.from_dict(dictionary_of_data_frames, orient='minor')
oriented_panel

<class 'pandas.core.panel.Panel'>
Dimensions: 3 (items) x 4 (major_axis) x 2 (minor_axis)
Items axis: 0 to 2
Major_axis axis: 0 to 3
Minor_axis axis: Item1 to Item2

### Panels using remote stock data
source: [Python for Data Analysis by Wes McKinney](http://shop.oreilly.com/product/0636920023784.do) page 152; modified by Charles Kelly to use DataReader

In [6]:
start = datetime.datetime(2010, 1, 1)
end = datetime.datetime(2016, 7, 15)
pdata = pd.Panel(dict((stk, data.DataReader("F", 'yahoo', start, end))
for stk in ['AAPL', 'GOOG', 'MSFT', 'DELL']))

pdata

<class 'pandas.core.panel.Panel'>
Dimensions: 4 (items) x 1645 (major_axis) x 6 (minor_axis)
Items axis: AAPL to MSFT
Major_axis axis: 2010-01-04 00:00:00 to 2016-07-15 00:00:00
Minor_axis axis: Open to Adj Close

##### swap axes: make the stocks the columns
documentation: http://pandas.pydata.org/pandas-docs/version/0.18.0/generated/pandas.Panel.swapaxes.html

In [7]:
pdata = pdata.swapaxes('items', 'minor')

In [8]:
pdata['Adj Close'].head()

AAPL  DELL  GOOG  MSFT
Date                              
2010-01-04  8.66  8.66  8.66  8.66
2010-01-05  9.23  9.23  9.23  9.23
2010-01-06  9.58  9.58  9.58  9.58
2010-01-07  9.82  9.82  9.82  9.82
2010-01-08  9.85  9.85  9.85  9.85

##### NumPy ix generalizes to three dimensions
source: http://docs.scipy.org/doc/numpy/reference/generated/numpy.ix_.html

In [9]:
pdata.ix[:, '7/12/2016', :]

Open   High    Low  Close  Volume  Adj Close
AAPL  13.38  13.54  13.38  13.45  31.32M      13.30
DELL  13.38  13.54  13.38  13.45  31.32M      13.30
GOOG  13.38  13.54  13.38  13.45  31.32M      13.30
MSFT  13.38  13.54  13.38  13.45  31.32M      13.30

##### convert Panel to DataFrame with multi-index

In [10]:
stacked = pdata.ix[:, '6/30/2016':, :].to_frame()
print(type(stacked))
stacked

<class 'pandas.core.frame.DataFrame'>


Open   High    Low  Close  Volume  Adj Close
Date       minor                                               
2016-06-30 AAPL   12.56  12.57  12.26  12.57  44.98M      12.43
           DELL   12.56  12.57  12.26  12.57  44.98M      12.43
           GOOG   12.56  12.57  12.26  12.57  44.98M      12.43
           MSFT   12.56  12.57  12.26  12.57  44.98M      12.43
2016-07-01 AAPL   12.88  12.89  12.69  12.72  45.07M      12.58
           DELL   12.88  12.89  12.69  12.72  45.07M      12.58
           GOOG   12.88  12.89  12.69  12.72  45.07M      12.58
           MSFT   12.88  12.89  12.69  12.72  45.07M      12.58
2016-07-05 AAPL   12.57  12.59  12.31  12.40  30.24M      12.27
           DELL   12.57  12.59  12.31  12.40  30.24M      12.27
           GOOG   12.57  12.59  12.31  12.40  30.24M      12.27
           MSFT   12.57  12.59  12.31  12.40  30.24M      12.27
2016-07-06 AAPL   12.32  12.58  12.21  12.57  35.09M      12.43
           DELL   12.32  12.58  12.21  12.57  35.09M      12.43
           GOOG   12.32  12.58  12.21  12.57  35.09M      12.43
           MSFT   12.32  12.58  12.21  12.57  35.09M      12.43
2016-07-07 AAPL   12.57  12.84  12.57  12.75  33.47M      12.61
           DELL   12.57  12.84  12.57  12.75  33.47M      12.61
           GOOG   12.57  12.84  12.57  12.75  33.47M      12.61
           MSFT   12.57  12.84  12.57  12.75  33.47M      12.61
2016-07-08 AAPL   12.89  13.17  12.89  13.09  45.95M      12.95
           DELL   12.89  13.17  12.89  13.09  45.95M      12.95
           GOOG   12.89  13.17  12.89  13.09  45.95M      12.95
           MSFT   12.89  13.17  12.89  13.09  45.95M      12.95
2016-07-11 AAPL   13.17  13.33  13.13  13.27  28.63M      13.13
           DELL   13.17  13.33  13.13  13.27  28.63M      13.13
           GOOG   13.17  13.33  13.13  13.27  28.63M      13.13
           MSFT   13.17  13.33  13.13  13.27  28.63M      13.13
2016-07-12 AAPL   13.38  13.54  13.38  13.45  31.32M      13.30
           DELL   13.38  13.54  13.38  13.45  31.32M      13.30
           GOOG   13.38  13.54  13.38  13.45  31.32M      13.30
           MSFT   13.38  13.54  13.38  13.45  31.32M      13.30
2016-07-13 AAPL   13.49  13.51  13.31  13.48  26.78M      13.33
           DELL   13.49  13.51  13.31  13.48  26.78M      13.33
           GOOG   13.49  13.51  13.31  13.48  26.78M      13.33
           MSFT   13.49  13.51  13.31  13.48  26.78M      13.33
2016-07-14 AAPL   13.57  13.70  13.55  13.59  31.80M      13.44
           DELL   13.57  13.70  13.55  13.59  31.80M      13.44
           GOOG   13.57  13.70  13.55  13.59  31.80M      13.44
           MSFT   13.57  13.70  13.55  13.59  31.80M      13.44
2016-07-15 AAPL   13.63  13.67  13.53  13.57  19.55M      13.42
           DELL   13.63  13.67  13.53  13.57  19.55M      13.42
           GOOG   13.63  13.67  13.53  13.57  19.55M      13.42
           MSFT   13.63  13.67  13.53  13.57  19.55M      13.42

##### convert DataFrame with multi-index to a Panel

In [11]:
stacked.to_panel()

<class 'pandas.core.panel.Panel'>
Dimensions: 6 (items) x 11 (major_axis) x 4 (minor_axis)
Items axis: Open to Adj Close
Major_axis axis: 2016-06-30 00:00:00 to 2016-07-15 00:00:00
Minor_axis axis: AAPL to MSFT